In [1]:
# Imports
from datetime import datetime, timedelta, timezone
import dateutil
import pytz
import zoneinfo

import pendulum
import arrow
import whenever

**1. Test cases:**

0: Normal. Should be 18 hours.

1: Skip an hour that night. Should be 17 hours.

2: Go back an hour that night. Should be 19 hours.

3: Skip an hour this morning. Should be 39 hours.

4: Before going back an hour that morning. Should be 41 hours.

5: Within DST hour 1st time. Should be 40 hours 30 minutes.

6: Within DST hour 2nd time. Should be 39 hours 30 minutes.

7: Tm at this time is during DST 1st time. Should be 40 hours 30 minutes.

8: Tm at this time is during DST 2nd time. Should be 39 hours 30 minutes.

In [100]:
# Finding list of leap seconds (unused)

#  from: https://hpiers.obspm.fr/iers/bul/bulc/Leap_Second.dat

text='''1,1,1972
1,7,1972
1,1,1973
1,1,1974
1,1,1975
1,1,1976
1,1,1977
1,1,1978
1,1,1979
1,1,1980
1,7,1981
1,7,1982
1,7,1983
1,7,1985
1,1,1988
1,1,1990
1,1,1991
1,7,1992
1,7,1993
1,7,1994
1,1,1996
1,7,1997
1,1,1999
1,1,2006
1,1,2009
1,7,2012
1,7,2015
1,1,2017'''
separated = text.split("\n")

leap_second_dates = []

for val in separated:
        date = val.split(",")
        leap_second_dates.append(datetime(int(date[2]), int(date[1]), int(date[0]), tzinfo=dateutil.tz.UTC))


In [2]:
# Defining test cases:

du_tzNYC = dateutil.tz.gettz("America/New_York")

tcs = [
        (datetime(2024, 6, 7, 23), False, 18 * 3600), # 0: Normal. Should be 18 hours.
        (datetime(2022, 3, 12, 23), False, 17 * 3600), # 1: Skip an hour that night. Should be 17 hours.
        (datetime(2022, 11, 5, 23), False, 19 * 3600), # 2: Go back an hour that night. Should be 19 hours.
        (datetime(2022, 3, 13, 1), False, 39 * 3600), # 3: Skip an hour this morning. Should be 39 hours.
        (datetime(2022, 11, 6, 1, fold=0), False, 41 * 3600), # 4: Before going back an hour that morning. Should be 41 hours.
        (datetime(2022, 11, 6, 1, 30, fold=0), False, 40 * 3600 + 30 * 60), # 5: Within DST hour 1st time. Should be 40 hours 30 minutes.
        (datetime(2022, 11, 6, 1, 30, fold=1), False, 39 * 3600 + 30 * 60), # 6: Within DST hour 2nd time. Should be 39 hours 30 minutes.
        (datetime(2022, 11, 5, 1, 30), False, 40 * 3600 + 30 * 60), # 7: Tm at this time is during DST 1st time. Should be 40 hours 30 minutes.
        (datetime(2022, 11, 5, 2, 30), False, 39 * 3600 + 30 * 60), # 8: Tm at this time is during DST 2nd time. Should be 39 hours 30 minutes.
        # (datetime(2015, 6, 30, 23), False, 18 * 3600 + 1), # 9: Leap second that day. Should be 18 hours 1 second.
        # (datetime(2015, 6, 30, 23, 59, 59, int(1e6) // 2), False, 17 * 3600 + 1.5), # 10: Within a leap second the 1st time. Should be 17 hours 1.5 seconds.
        # (datetime(2015, 6, 30, 23, 59, 59, int(1e6) // 2), True, 17 * 3600 + 0.5) # 11: Within a leap second the 2nd time. Should be 17 hours 0.5 seconds.
]

tcs_pendulum = [
        (pendulum.datetime(2024, 6, 7, 23, tz="America/New_York"), False, 18 * 3600), # 0: Normal. Should be 18 hours.
        (pendulum.datetime(2022, 3, 12, 23, tz="America/New_York"), False, 17 * 3600), # 1: Skip an hour that night. Should be 17 hours.
        (pendulum.datetime(2022, 11, 5, 23, tz="America/New_York"), False, 19 * 3600), # 2: Go back an hour that night. Should be 19 hours.
        (pendulum.datetime(2022, 3, 13, 1, tz="America/New_York"), False, 39 * 3600), # 3: Skip an hour this morning. Should be 39 hours.
        (pendulum.datetime(2022, 11, 6, 1, dst_rule=pendulum.PRE_TRANSITION, tz="America/New_York"), False, 41 * 3600), # 4: Before going back an hour that morning. Should be 41 hours.
        (pendulum.datetime(2022, 11, 6, 1, 30, dst_rule=pendulum.PRE_TRANSITION, tz="America/New_York"), False, 40 * 3600 + 30 * 60), # 5: Within DST hour 1st time. Should be 40 hours 30 minutes.
        (pendulum.datetime(2022, 11, 6, 1, 30, dst_rule=pendulum.POST_TRANSITION, tz="America/New_York"), False, 39 * 3600 + 30 * 60), # 6: Within DST hour 2nd time. Should be 39 hours 30 minutes.
        (pendulum.datetime(2022, 11, 5, 1, 30, tz="America/New_York"), False, 40 * 3600 + 30 * 60), # 7: Tm at this time is during DST 1st time. Should be 40 hours 30 minutes.
        (pendulum.datetime(2022, 11, 5, 2, 30, tz="America/New_York"), False, 39 * 3600 + 30 * 60), # 8: Tm at this time is during DST 2nd time. Should be 39 hours 30 minutes.
        # (pendulum.datetime(2015, 6, 30, 23, tz="America/New_York"), False, 18 * 3600 + 1), # 9: Leap second that day. Should be 18 hours 1 second.
        # (pendulum.datetime(2015, 6, 30, 23, 59, 59, int(1e6) // 2, tz="America/New_York"), False, 17 * 3600 + 1.5), # 10: Within a leap second the 1st time. Should be 17 hours 1.5 seconds.
        # (pendulum.datetime(2015, 6, 30, 23, 59, 59, int(1e6) // 2, tz="America/New_York"), True, 17 * 3600 + 0.5) # 11: Within a leap second the 2nd time. Should be 17 hours 0.5 seconds.
]

tcs_arrow = [
        (arrow.get(2024, 6, 7, 23, tzinfo=du_tzNYC), False, 18 * 3600), # 0: Normal. Should be 18 hours.
        (arrow.get(2022, 3, 12, 23, tzinfo=du_tzNYC), False, 17 * 3600), # 1: Skip an hour that night. Should be 17 hours.
        (arrow.get(2022, 11, 5, 23, tzinfo=du_tzNYC), False, 19 * 3600), # 2: Go back an hour that night. Should be 19 hours.
        (arrow.get(2022, 3, 13, 1, tzinfo=du_tzNYC), False, 39 * 3600), # 3: Skip an hour this morning. Should be 39 hours.
        (arrow.get(2022, 11, 6, 1, tzinfo=du_tzNYC), False, 41 * 3600), # 4: Before going back an hour that morning. Should be 41 hours.
        (arrow.get(2022, 11, 6, 1, 30, tzinfo=du_tzNYC, fold=0), False, 40 * 3600 + 30 * 60), # 5: Within DST hour 1st time. Should be 40 hours 30 minutes.
        (arrow.get(2022, 11, 6, 1, 30, tzinfo=du_tzNYC, fold=1), False, 39 * 3600 + 30 * 60), # 6: Within DST hour 2nd time. Should be 39 hours 30 minutes.
        (arrow.get(2022, 11, 5, 1, 30, tzinfo=du_tzNYC), False, 40 * 3600 + 30 * 60), # 7: Tm at this time is during DST 1st time. Should be 40 hours 30 minutes.
        (arrow.get(2022, 11, 5, 2, 30, tzinfo=du_tzNYC), False, 39 * 3600 + 30 * 60), # 8: Tm at this time is during DST 2nd time. Should be 39 hours 30 minutes.
        # (arrow.get(2015, 6, 30, 23), False, 18 * 3600 + 1), # 9: Leap second that day. Should be 18 hours 1 second.
        # (arrow.get(2015, 6, 30, 23, 59, 59, int(1e6) // 2), False, 17 * 3600 + 1.5), # 10: Within a leap second the 1st time. Should be 17 hours 1.5 seconds.
        # (arrow.get(2015, 6, 30, 23, 59, 59, int(1e6) // 2), True, 17 * 3600 + 0.5) # 11: Within a leap second the 2nd time. Should be 17 hours 0.5 seconds.
]

tcs_whenever = [
        (whenever.ZonedDateTime(2024, 6, 7, 23, tz="America/New_York"), False, 18 * 3600), # 0: Normal. Should be 18 hours.
        (whenever.ZonedDateTime(2022, 3, 12, 23, tz="America/New_York"), False, 17 * 3600), # 1: Skip an hour that night. Should be 17 hours.
        (whenever.ZonedDateTime(2022, 11, 5, 23, tz="America/New_York"), False, 19 * 3600), # 2: Go back an hour that night. Should be 19 hours.
        (whenever.ZonedDateTime(2022, 3, 13, 1, tz="America/New_York"), False, 39 * 3600), # 3: Skip an hour this morning. Should be 39 hours.
        (whenever.ZonedDateTime(2022, 11, 6, 1, tz="America/New_York", disambiguate="earlier"), False, 41 * 3600), # 4: Before going back an hour that morning. Should be 41 hours.
        (whenever.ZonedDateTime(2022, 11, 6, 1, 30, tz="America/New_York", disambiguate="earlier"), False, 40 * 3600 + 30 * 60), # 5: Within DST hour 1st time. Should be 40 hours 30 minutes.
        (whenever.ZonedDateTime(2022, 11, 6, 1, 30, tz="America/New_York", disambiguate="later"), False, 39 * 3600 + 30 * 60), # 6: Within DST hour 2nd time. Should be 39 hours 30 minutes.
        (whenever.ZonedDateTime(2022, 11, 5, 1, 30, tz="America/New_York"), False, 40 * 3600 + 30 * 60), # 7: Tm at this time is during DST 1st time. Should be 40 hours 30 minutes.
        (whenever.ZonedDateTime(2022, 11, 5, 2, 30, tz="America/New_York"), False, 39 * 3600 + 30 * 60), # 8: Tm at this time is during DST 2nd time. Should be 39 hours 30 minutes.
        # (whenever.ZonedDateTime(2015, 6, 30, 23), False, 18 * 3600 + 1), # 9: Leap second that day. Should be 18 hours 1 second.
        # (whenever.ZonedDateTime(2015, 6, 30, 23, 59, 59, int(1e6) // 2), False, 17 * 3600 + 1.5), # 10: Within a leap second the 1st time. Should be 17 hours 1.5 seconds.
        # (whenever.ZonedDateTime(2015, 6, 30, 23, 59, 59, int(1e6) // 2), True, 17 * 3600 + 0.5) # 11: Within a leap second the 2nd time. Should be 17 hours 0.5 seconds.
]

**Above are the testcases formatted for each individual library ^^^**
\
\*\* The timezone providers share the same set of testcases (using the default DateTime library) 

In [12]:
# DateTime:

def calc_diff(td, leap_second_fold, tzNYC, tzUTC, tzp) -> tuple[float, str]:
        dbg_str = ""
        dbg_cnt = 0
        dbg_str += f"{dbg_cnt}: tzp: {tzp}\n"; dbg_cnt += 1
        # dbg_str += f"{dbg_cnt}: td: {td}\n"; dbg_cnt += 1
        
        td_localized = td
        if tzp == "pytz":
                td_localized = tzNYC.localize(td_localized)
        else:
                td_localized = td_localized.astimezone(tzNYC)
        # dbg_str += f"{dbg_cnt}: td_localized: {td_localized}\n"; dbg_cnt += 1
        
        tm = td_localized + timedelta(days=1)
        dbg_str += f"{dbg_cnt}: tm: {tm}\n"; dbg_cnt += 1
        
        # I'm going to assume that no trickery will occur where a \
        # daylight savings will change time over more than 12 hours. \
        # Without this, we can actually skip an additional day on \
        # accident if the hour is >= 23 in the case of pytz
        
        # tm_norm = datetime(tm.year, tm.month, tm.day, 12, tzinfo=tm.tzinfo)
        tm_norm = tm
        
        if tzp == "pytz":
                tm_norm = tm_norm.astimezone(tzNYC)
        dbg_str += f"{dbg_cnt}: tm_norm: {tm_norm}\n"; dbg_cnt += 1
        
        tm_five = tm_norm.replace(hour=17)
        dbg_str += f"{dbg_cnt}: tm_five: {tm_five}\n"; dbg_cnt += 1

        diff = (tm_five.astimezone(tzUTC) - td.astimezone(tzUTC)).total_seconds()
        dbg_str += f"{dbg_cnt}: diff: {diff}\n"; dbg_cnt += 1

        return diff, dbg_str

# I will assume that we are in "America/New_York" since there \ 
# does not appear to be a consistent way to get access to the \
# full timezone name (e.g., not just "EDT" but the whole string)

du_tzNYC = dateutil.tz.gettz("America/New_York")
du_tzUTC = dateutil.tz.gettz("UTC")
pytz_tzNYC = pytz.timezone("America/New_York")
pytz_tzUTC = pytz.timezone("UTC")
zi_tzNYC = zoneinfo.ZoneInfo("America/New_York")
zi_tzUTC = zoneinfo.ZoneInfo("UTC")
# 
# I will refrain from using timezone since it will clearly not work. \
# Unless I keep track of the daylight savings shifts myself I don't \
# see a way of keeping the timedeltas correct.
# 
# dt_tzNYC = timezone("America/New_York")
# dt_tzUTC = timezone(timedelta(hours=0))

timezones = {"pytz":(pytz_tzNYC, pytz_tzUTC),
             "dateutil.tz":(du_tzNYC, du_tzUTC),
             "zoneinfo":(zi_tzNYC, zi_tzUTC),
        #      "datetime.timezone":(dt_tzNYC, dt_tzUTC)
             }

error = False

for tzp in timezones.keys():
        for index, tc in enumerate(tcs):
                calculated_diff, dbg_str = calc_diff(tc[0], tc[1], timezones[tzp][0], timezones[tzp][1], tzp)
                E_diff = tc[2]
                if calculated_diff != E_diff:
                        print(dbg_str[:-1])
                        print(f"Error: {tzp} @ {index}: {calculated_diff} should be {E_diff}")
                        print()
                        error = True
        
if (not error):
        print("No TC errors found")

# diff, _ = calc_diff(datetime.now(), False, du_tzNYC, du_tzUTC, "dateutil.tz")
# print(f"\nFrom right now: {int(diff / 3600)}h {int(diff % 3600 / 60)}m {diff % 60}s")

0: tzp: pytz
1: tm: 2022-03-13 23:00:00-05:00
2: tm_norm: 2022-03-14 00:00:00-04:00
3: tm_five: 2022-03-14 17:00:00-04:00
4: diff: 147600.0
Error: pytz @ 1: 147600.0 should be 61200

0: tzp: pytz
1: tm: 2022-11-07 01:30:00-05:00
2: tm_norm: 2022-11-07 01:30:00-05:00
3: tm_five: 2022-11-07 17:30:00-05:00
4: diff: 147600.0
Error: pytz @ 5: 147600.0 should be 145800

0: tzp: pytz
1: tm: 2022-11-07 01:30:00-05:00
2: tm_norm: 2022-11-07 01:30:00-05:00
3: tm_five: 2022-11-07 17:30:00-05:00
4: diff: 144000.0
Error: pytz @ 6: 144000.0 should be 142200

0: tzp: pytz
1: tm: 2022-11-06 01:30:00-04:00
2: tm_norm: 2022-11-06 01:30:00-04:00
3: tm_five: 2022-11-06 17:30:00-04:00
4: diff: 144000.0
Error: pytz @ 7: 144000.0 should be 145800

0: tzp: pytz
1: tm: 2022-11-06 02:30:00-04:00
2: tm_norm: 2022-11-06 01:30:00-05:00
3: tm_five: 2022-11-06 17:30:00-05:00
4: diff: 144000.0
Error: pytz @ 8: 144000.0 should be 142200

0: tzp: dateutil.tz
1: tm: 2022-11-07 01:30:00-05:00
2: tm_norm: 2022-11-07 01:30

**2-4. PYTZ is the only library that needs additional help above ^^^**

That is, we must explicitly set the time to 12PM before normalizing the timezone. This is surprising behavior.

**5. dateutil.tz automatically changes time zones as we move forwards and backwards in time.**

PYTZ, on the other hand, does not.

Thus, we need to check if any timezone changes have occurred and update accordingly using normalize() / astimezone().

\*\*(I'm not sure of the difference between normalize and astimezone)

**6. Below are the implementations for pendulum, arrow, and whenever**

They seem to work properly by default

In [4]:
# Pendulum:

def calc_diff_pendulum(td, leap_second_fold) -> tuple[float, str]:
        dbg_str = ""
        dbg_cnt = 0
        dbg_str += f"{dbg_cnt}: td: {td}\n"; dbg_cnt += 1
        tm = td.add(days=1)
        dbg_str += f"{dbg_cnt}: tm: {tm}\n"; dbg_cnt += 1
        tm_five = tm.at(17)
        dbg_str += f"{dbg_cnt}: tm_five: {tm_five}\n"; dbg_cnt += 1
        diff = (tm_five - td).total_seconds()
        return diff, dbg_str

error = False

for index, tc in enumerate(tcs_pendulum):
        calculated_diff, dbg_str = calc_diff_pendulum(tc[0], tc[1])
        E_diff = tc[2]
        if calculated_diff != E_diff:
                print(dbg_str[:-1])
                print(f"Error: {index}: {calculated_diff} should be {E_diff}")
                print()
                error = True

if (not error):
        print("No TC errors found")

No TC errors found


In [11]:
# Arrow:

def calc_diff_arrow(td, leap_second_fold) -> tuple[float, str]:
        dbg_str = ""
        dbg_cnt = 0
        dbg_str += f"{dbg_cnt}: td: {td}\n"; dbg_cnt += 1
        tm = td + timedelta(days=1)
        dbg_str += f"{dbg_cnt}: tm: {tm}\n"; dbg_cnt += 1
        tm_five = tm.replace(hour=17, minute=0, second=0, microsecond=0)
        dbg_str += f"{dbg_cnt}: tm_five: {tm_five}\n"; dbg_cnt += 1
        diff = (tm_five.astimezone(du_tzUTC) - td.astimezone(du_tzUTC)).total_seconds()
        # diff = (tm_five - td).total_seconds()
        return diff, dbg_str

error = False

for index, tc in enumerate(tcs_arrow):
        calculated_diff, dbg_str = calc_diff_arrow(tc[0], tc[1])
        E_diff = tc[2]
        if calculated_diff != E_diff:
                print(dbg_str[:-1])
                print(f"Error: {index}: {calculated_diff} should be {E_diff}")
                print()
                error = True

if (not error):
        print("No TC errors found")

No TC errors found


In [6]:
# Whenever:

def calc_diff_arrow(td, leap_second_fold) -> tuple[float, str]:
        dbg_str = ""
        dbg_cnt = 0
        dbg_str += f"{dbg_cnt}: td: {td}\n"; dbg_cnt += 1
        tm = td + whenever.days(1)
        dbg_str += f"{dbg_cnt}: tm: {tm}\n"; dbg_cnt += 1
        tm_five = tm.replace(hour=17, minute=0, second=0, microsecond=0)
        dbg_str += f"{dbg_cnt}: tm_five: {tm_five}\n"; dbg_cnt += 1
        diff = (tm_five - td).in_seconds()
        return diff, dbg_str

error = False

for index, tc in enumerate(tcs_whenever):
        calculated_diff, dbg_str = calc_diff_arrow(tc[0], tc[1])
        E_diff = tc[2]
        if calculated_diff != E_diff:
                print(dbg_str[:-1])
                print(f"Error: {index}: {calculated_diff} should be {E_diff}")
                print()
                error = True

if (not error):
        print("No TC errors found")

No TC errors found
